In [1]:
import ee
import sys
import os

sys.path.append(os.path.abspath('../'))

from modules.SpectralIndexes import *
from modules.Miscellaneous import *
from modules.Mosaic import *
from modules.SmaAndNdfi import * 
from modules import Map

ee.Initialize()


In [2]:
regionAsset = 'projects/mapbiomas-workspace/AUXILIAR/landsat-scenes'

## define geometry
pathRow = '217073';

geometry = ee.Feature(
    ee.FeatureCollection(regionAsset)
        .filterMetadata('WRSPR', 'equals', pathRow)
        .first()).geometry()


In [3]:
PARAMS = (
    # {'year': 2020, 'dateStart': '2020-01-01', 'dateEnd': '2020-12-31', 'territory': 'MATA ATLANTICA'},
    # {'year': 2021, 'dateStart': '2021-01-01', 'dateEnd': '2021-12-31', 'territory': 'MATA ATLANTICA'},
    # {'year': 2022, 'dateStart': '2022-01-01', 'dateEnd': '2022-12-31', 'territory': 'MATA ATLANTICA'},
    {'year': 2023, 'dateStart': '2023-01-01', 'dateEnd': '2023-12-31', 'territory': 'MATA ATLANTICA'},
    # {'year': 2024, 'dateStart': '2024-01-01', 'dateEnd': '2024-12-31', 'territory': 'MATA ATLANTICA'},
)


In [4]:
## Landsat collections
collectionId = 'LANDSAT/COMPOSITES/C02/T1_L2_32DAY'

## spectral bands selected
spectralBands = ['blue', 'red', 'green', 'nir', 'swir1', 'swir2']

## endemembers collection
endmembers = ENDMEMBERS['landsat-8']


In [5]:
# rgb layer
visParams = {"bands": "swir1_median,nir_median,red_median","gain": "0.08,0.06,0.2", "gamma": "0.65"}

In [6]:
for obj in PARAMS:
    collection = (ee.ImageCollection(collectionId)
                  .filter(ee.Filter.date(obj['dateStart'], obj['dateEnd']))
                  .filter(ee.Filter.bounds(geometry))
                  .select(spectralBands))

    # apply scaling factor
    collection = collection.map(
        lambda image: image.multiply(10000).copyProperties(
            image, ['system:time_start', 'system:time_end'])
    )

    # apply SMA
    collection = collection.map(
        lambda image: getFractions(image, endmembers)
    )

    # calculate SMA indexes
    collection = collection\
        .map(getNDFI)\
        .map(getSEFI)\
        .map(getWEFI)\
        .map(getFNS)

    # calculate Spectral indexes
    collection = collection\
        .map(getCAI)\
        .map(getEVI2)\
        .map(getGCVI)\
        .map(getHallCover)\
        .map(getHallHeigth)\
        .map(getNDVI)\
        .map(getNDWI)\
        .map(getPRI)\
        .map(getSAVI)

    mosaic = getMosaic(
        collection=collection,
        dateStart=obj['dateStart'],
        dateEnd=obj['dateEnd'],
        percentileBand='ndvi',
        percentileDry=25,
        percentileWet=75)

    # get other bands
    mosaic = getSlope(mosaic)
    mosaic = getEntropyG(mosaic)
    mosaic = mosaic.clip(geometry)


In [7]:
Map.addLayer(mosaic,
             visParams=visParams,
             name='mosaic')